In [ ]:
import panda as pd 
import numpy as np 
from keras.datasets import boston_housing
from keras import layers, models, optimizers

In [ ]:
(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

In [ ]:
train_data.shape

In [ ]:
train_labels.shape

In [ ]:
La unidades en las etiquetas(labels) están dividas entre 1000(1k)

In [ ]:
train_data[0]

In [ ]:
train_labels[0]

Se resta el promedio y se divide entre la desviación estandar para normalizar los datos

In [ ]:
mean = train_data.mean(axis = 0)
train_data = train_data - mean
std = train_data.std(axis=0)
train_data = train_data/std

# Por cuestiones de etiquetas, el modelo nunca debe saber otra información que no sea la del dataset
# de entrenamiento, por eso se le resta y se divide al dataset de prueba el mismo promedio y std
test_data = test_data - mean
test_data = test_data / std

In [ ]:
model = models.Sequential()

Mean Absolute Error es la métrica de éxito para este modelo de regression y el Mean Square Error como función de pérdida
En la última capa no se utiliza ningún tipo de función porque ser un modelo de regressión

In [ ]:
def build_model_regression(input_data,lr_var = 0.001):
    model.add(layers.Dense(64, activation = 'relu', input_shape = (input_data,)))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(1))
    model.compile(
            optimizers = optimizers.RMSprop(lr_var ),
            loss = 'mse',
            metrics = ['mae']
)



K-fold es un método para dividir los datos en diferente cantidades

In [ ]:
k = 4
num_val_samples = len(train_data) // k
num_epoch = 500
all_history = []

In [ ]:
for i in range(k):
    print('Fold:', i)
    val_data = train_data[i*num_val_samples : (i + 1) * num_val_samples]
    val_targets = train_labels[i*num_val_samples : (i + 1) * num_val_samples]

    partia_train_data = pd.concatenate([train_data[:i * num_val_samples],
                                        train_data[(i+1) * num_val_samples],
                                        ],
                                        axis = 0
                                        )
    partia_train_targets = pd.concatenate([train_labels[:i * num_val_samples],
                                        train_labels[(i+1) * num_val_samples],
                                        ],
                                        axis = 0
                                        )
    
    model = build_model_regression(0.001,13)

    history = model.fit(
        partia_train_data,
        partia_train_targets,
        epochs = num_epoch, 
        batch_size = 16,
        validation_data = (val_data,val_targets),
        verbose = 0
    )
    all_history.append(history.history['val_mae'])

Tiene 80 Resultados por la cantidad de épocas que tuvo

In [ ]:
len(all_history[0])

Creando un Dataframe con all_history y hayando el promedio

In [ ]:
all_mea_avg = pd.DataFrame(all_history).mean(axis = 0)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10,10))

plt.plot(range(1, len(all_mea_avg[15:]) + 1),all_mea_avg[15:])

In [ ]:
model.evaluate(test_data, test_labels)